# Differential Privacy for PRETSA

In [ ]:
!git clone https://github.com/cmalvr/PRETSA

In [ ]:
!pip install -r /content/PRETSA/requirements.txt

In [ ]:
!python /content/PRETSA/dataset_collection.py

## Running Annotations for three datasets

In [ ]:
# Generates all the original annotation duration for each of two datasets: CoSeLog and Sepsis
# Generates /content/PRETSA/original_annotation/ directory

!python /content/PRETSA/gc_add_annotations_duration.py

## Run Pretsa: Generates Pretsa Log


In [6]:
t = 0.1

In [ ]:
# Generates: /content/PRETSA/pretsalog/ f"{dataset}_t{t}_k{k}_pretsa.csv"
# These contain the new annotations for duration after Running PRETSA method

!python /content/PRETSA/startExperimentsForJournalExtension_PRETSA.py {t}

## Run Pretsa Binary: Generates Pretsa Binary Log

In [ ]:
# Generates: /content/PRETSA/pretsa_binarylog/ f"{dataset}_t{t}_k{k}_pretsa.csv"
# These contain the new annotations for duration after Running PRETSA method

!python /content/PRETSA/startExperimentsForJournalExtension_PRETSA.py {t}

## Run Baseline: Generate Baseline log

In [ ]:
# Generates: /content/PRETSA/baselinelogs/ Path(filePath).name.replace(".csv","_pretsa_baseline_k%s_t%s.csv" % (kString,tString))
# These contain the new annotations for duration after Running BASELINE method
# Should generate same content as baselogs zip directory

!python /content/PRETSA/gc_generate_baseline_log.py {t}

## Calculate Statistics

In [ ]:
#Generates: "baseline_event_logs_statistics/"
#!python /content/PRETSA/calculateBaselineEventLogStatistics.py {t}

#Generates: "PRETSA_event_logs_statistics/"
#!python /content/PRETSA/calculatePRETSAEventLogStatistics.py  {t}

## Generate Annotations

In [10]:
 #Generates: dictPath + "pretsa_baseline_statistics_annotations.csv"
#!python /content/PRETSA/calculateAnnotationsEventLog_baseline.py {t}

#!python /content/PRETSA/calculateAnnotationsEventLog_pretsa.py {t}

#Generates original_annotations.csv
#!python /content/PRETSA/generateAnnotationOriginalDataset.py

## Calculate Annotation Errors

 Relative error of the annotations for each activity

In [ ]:
from calculateDurationDifferences import calculate_duration_differences
from mapDurationDifferences import generate_heatmap

# Replace with dataset name
error_data = calculate_duration_differences("CoSeLoG")
generate_heatmap("CoSeLoG")